In [ ]:
# Install packages if not installed yet (uncomment to run)
# !pip install pyomo pandas openpyxl
!apt-get install -y glpk-utils

from pyomo.environ import *
from google.colab import files

import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, ConstraintList, Expression, Binary, NonNegativeIntegers, maximize, SolverFactory
from google.colab import files

# Upload your dataset
uploaded = files.upload()

# Keep this as your working dataset
data = pd.read_excel('data_numeric_python_final1.xlsx')


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


Saving data_numeric_python_final1.xlsx to data_numeric_python_final1.xlsx


In [ ]:
print(data.isnull().sum())

total_gmv_usd               0
followers                   0
live_gmv_usd                0
go_live_num                 0
live_watch_pv               0
video_gmv_usd               0
publish_video_num           0
video_vv                    0
showcase_gmv_usd            0
video_reel_booking_price    0
livebooking_cost            0
live_duration_min           0
live_engagement             0
video_engagement            0
unit_livebooking_cost       0
unit_videobooking_cost      0
total_cost                  0
total_livebooking_cost      0
total_videobooking_cost     0
new_total_cost              0
tier_base_on_total_gmv      0
log_total_gmv_usd           0
predicted_log_gmv           0
residuals_log               0
new_predicted_gmv           0
kol_name                    0
dtype: int64


In [ ]:
print("kol_name" in data.columns)

True


In [ ]:

# Define M constant
M = 10000

# Number of KOLs
N = range(len(data))

# Initialize model
model = ConcreteModel()

# Decision variables
model.kol = Var(N, domain=Binary)
model.num_live = Var(N, domain=NonNegativeIntegers)
model.num_video = Var(N, domain=NonNegativeIntegers)

# Objective function: maximize total predicted GMV
model.obj = Objective(
    expr=sum(
        model.kol[i] * data['new_predicted_gmv'][i] +
        0.001747 * model.num_live[i] +
        0.001015 * model.num_video[i]
        for i in N
    ),
    sense=maximize
)

# Total budget expression
model.total_cost = Expression(
    expr=sum(
        model.num_live[i] * data['unit_livebooking_cost'][i] +
        model.num_video[i] * data['unit_videobooking_cost'][i]
        for i in N
    )
)

# Budget constraint
model.budget_constraint = Constraint(expr=model.total_cost <= 7500)

# Live cost constraint (<= 40% of total cost)
model.live_cost_constraint = Constraint(
    expr=sum(
        model.num_live[i] * data['unit_livebooking_cost'][i]
        for i in N
    ) >= 0.4 * model.total_cost
)

# Video cost constraint (>= 10% of total cost)
model.video_cost_constraint = Constraint(
    expr=sum(
        model.num_video[i] * data['unit_videobooking_cost'][i]
        for i in N
    ) >= 0.1 * model.total_cost
)

# Engagement constraint
model.engagement_constraint = Constraint(
    expr=sum(
        model.num_live[i] * data['live_engagement'][i] +
        model.num_video[i] * data['video_engagement'][i]
        for i in N
    ) >= 4000000
)

# Create binary indicators for each tier
is_mega = (data['tier_base_on_total_gmv'] == "Mega").astype(int)
is_macro = (data['tier_base_on_total_gmv'] == "Macro").astype(int)
is_micro = (data['tier_base_on_total_gmv'] == "Micro").astype(int)

# Add Logical Constraints
model.logical_constraints = ConstraintList()
for i in N:
    # Basic selection logic
    model.logical_constraints.add(model.num_live[i] <= M * model.kol[i])
    model.logical_constraints.add(model.num_video[i] <= M * model.kol[i])

    # Must do at least 1 of each if selected
    model.logical_constraints.add(model.num_live[i] >= model.kol[i])
    model.logical_constraints.add(model.num_video[i] >= model.kol[i])


    # Max 5 lives/videos per KOL
    model.logical_constraints.add(model.num_live[i] <= 5 * model.kol[i])
    model.logical_constraints.add(model.num_video[i] <= 5 * model.kol[i])



# Require at least one Mega KOL selected
model.require_mega = Constraint(expr=sum(model.kol[i] * is_mega[i] for i in N) >= 1)

# Require at least one Macro KOL selected
model.require_macro = Constraint(expr=sum(model.kol[i] * is_macro[i] for i in N) >= 1)

# Require at least one Micro KOL selected
model.require_micro = Constraint(expr=sum(model.kol[i] * is_micro[i] for i in N) >= 1)



from pyomo.opt import SolverFactory

# Create solver
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
solver.options['tmlim'] = 300
# Solve the model
results = solver.solve(model, tee=True)

# Check solver status
print(f"Solver status: {results.solver.status}")
print(f"Solver termination condition: {results.solver.termination_condition}")

# Extract results
selected = []
for i in N:
    if model.kol[i].value == 1:
        selected.append({
            'kol_name': data.loc[i, 'kol_name'],
            'Selected': int(model.kol[i].value),
            'Num_Live': int(model.num_live[i].value),
            'Num_Video': int(model.num_video[i].value)
        })

# Save selected KOLs to Excel
results_df = pd.DataFrame(selected)
results_df.to_excel('selected_kols_bonmin.xlsx', index=False)

print("✅ Optimization completed! Selected KOLs saved to 'selected_kols_bonmin.xlsx'.")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 300 --write /tmp/tmp1w4mlw5t.glpk.raw --wglp /tmp/tmptpwraoit.glpk.glp
 --cpxlp /tmp/tmppk1dpf5t.pyomo.lp
Reading problem data from '/tmp/tmppk1dpf5t.pyomo.lp'...
/tmp/tmppk1dpf5t.pyomo.lp:92504: warning: lower bound of variable 'x2' redefined
/tmp/tmppk1dpf5t.pyomo.lp:92504: warning: upper bound of variable 'x2' redefined
11929 rows, 5961 columns, 40810 non-zeros
5961 integer variables, 1987 of which are binary
94491 lines were read
Writing problem data to '/tmp/tmptpwraoit.glpk.glp'...
80567 lines were written
GLPK Integer Optimizer 5.0
11929 rows, 5961 columns, 40810 non-zeros
5961 integer variables, 1987 of which are binary
Preprocessing...
24 hidden packing inequaliti(es) were detected
4117 constraint coefficient(s) were reduced
11929 rows, 5961 columns, 40810 non-zeros
5961 integer variables, 1999 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  8.194e+06  ratio =  8.194e+06
G

In [ ]:
# View selected KOLs
results_df

,kol_name,Selected,Num_Live,Num_Video
0,khanhtrinh53,1,1,0
1,chienthanreview28,1,1,0
2,chungocanh68,1,1,0
3,chucareviewkhongbooking,1,1,0
4,hothao.12,1,1,0
5,bepcuagiao,1,1,0
6,tienkiemchung,1,1,0
7,qdhdangiukiute,1,1,0
8,22nastore_,1,1,0
9,uninyeudau,1,1,0


In [ ]:
from google.colab import files
# Download the Excel file to your computer
files.download('selected_kols_bonmin.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from pyomo.environ import value

final_objective_value = value(model.obj)
print(f"Final Objective Value = {final_objective_value:,.2f} USD")

Final Objective Value = 387,000.22 USD


In [ ]:
# Simply evaluate the objective expression after solving
final_objective_value = model.obj.expr()

print(f"Final Objective Value = {final_objective_value:,.2f} USD")

Final Objective Value = 387,000.22 USD


In [ ]:
# Merge selection results with original data
results_df_full = results_df.merge(data, on='kol_name', how='left')

# Calculate individual costs
results_df_full['live_cost'] = results_df_full['Num_Live'] * results_df_full['unit_livebooking_cost']
results_df_full['video_cost'] = results_df_full['Num_Video'] * results_df_full['unit_videobooking_cost']

# Calculate total expenses
total_live_expense = results_df_full['live_cost'].sum()
total_video_expense = results_df_full['video_cost'].sum()
total_expense = total_live_expense + total_video_expense

# Show the results
print("✅ Expenses based on selected KOLs:")
print(f"Total Live Expense: ${total_live_expense:,.2f}")
print(f"Total Video Expense: ${total_video_expense:,.2f}")
print(f"Total Overall Expense: ${total_expense:,.2f}")

✅ Expenses based on selected KOLs:
Total Live Expense: $3,042.50
Total Video Expense: $4,452.50
Total Overall Expense: $7,495.00


In [ ]:
print(data.columns)


Index(['total_gmv_usd', 'followers', 'live_gmv_usd', 'go_live_num',
       'live_watch_pv', 'video_gmv_usd', 'publish_video_num', 'video_vv',
       'showcase_gmv_usd', 'video_reel_booking_price', 'livebooking_cost',
       'live_duration_min', 'live_engagement', 'video_engagement',
       'unit_livebooking_cost', 'unit_videobooking_cost', 'total_cost',
       'total_livebooking_cost', 'total_videobooking_cost', 'new_total_cost',
       'tier_base_on_total_gmv', 'predicted_gmv_excluded_vars', 'kol_name'],
      dtype='object')


In [ ]:
# Merge the selection results with the original data
results_df_full = results_df.merge(data, on='kol_name', how='left')

# Step 1: Calculate engagement contribution
results_df_full['live_engagement_contribution'] = results_df_full['Num_Live'] * results_df_full['live_engagement']
results_df_full['video_engagement_contribution'] = results_df_full['Num_Video'] * results_df_full['video_engagement']

# Step 2: Total engagement achieved
total_engagement = results_df_full['live_engagement_contribution'].sum() + results_df_full['video_engagement_contribution'].sum()

# Step 3: Print result
print(f"Total Engagement Achieved: {total_engagement:,.0f}")


Total Engagement Achieved: 4,850,102


In [ ]:
# First, create an empty list to store the predicted GMV for selected KOLs
selected_predicted_gmv = []

# Loop through all KOLs
for i in N:
    if model.kol[i].value == 1:  # If this KOL was selected
        selected_predicted_gmv.append({
            'kol_name': data.loc[i, 'kol_name'],
            'new_predicted_gmv': data.loc[i, 'new_predicted_gmv']
        })

# Convert to a DataFrame for easy viewing
selected_predicted_gmv_df = pd.DataFrame(selected_predicted_gmv)

# Show the result
print(selected_predicted_gmv_df)

# (Optional) Save to Excel
selected_predicted_gmv_df.to_excel('selected_predicted_gmv.xlsx', index=False)

# (Optional) Download to your computer
from google.colab import files
files.download('selected_predicted_gmv.xlsx')

                  kol_name  new_predicted_gmv
0  chucareviewkhongbooking       70254.959034
1               cohanhacua       46683.745651
2            nganngocnga99       46938.704789
3               miho.house       28876.311923
4              olayvietnam       10457.784213
5         crazyteen.studio       74256.908688
6            kozoda_review       56187.002560
7               ng.bc.polo       18076.976034
8            winter_79.008       16872.723766
9      chiasebiquyetlamdep       18395.079097


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>